# Assignment 4 - Neural Network

## Grade: 100 pts + 10 Bonus 

This notebook contains the questions for Assignment 4. 

You must upload this completed Jupyter Notebook file as your submission (other file types are not permitted and will result in a grade of 0).***

* If you have trouble running neural network models on your laptop, you can use online platforms, like **[Google Colab](https://colab.research.google.com/)**.
* All Figures should have a x- and y-axis label and an appropriate title.
**Ensure that your code runs correctly by choosing "Kernel -> Restart and Cell -> Run All" before submitting.**

In [ ]:
# You are allowed to use other libraries as needed

import warnings 
warnings.filterwarnings('ignore')

import time as t
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
from sklearn.preprocessing import StandardScaler

from keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Dense, Input 
import tensorflow as tf

import time

#add other imports here if any (for example, pytorch)
from keras.optimizers import SGD

## Data set 
As modern vehicles have lots of connectivity, protecting in-vehicle network from cyber-attacks is an important issue. Controller Area Network (CAN) is the standard protocol for the in-vehicle network. But, the lack of security features in the CAN protocol makes vehicles vulnerable to attacks. The message injection attack is a representative attack type which injects fabricated messages to deceive electronic control unit (ECUs) or cause malfunctions. Through this notebook, you will develop ML modeles to detect different types of CAN attacks and protect vehicle networks. 

### Source
The dataset (CAN-intrusion-dataset-10000.csv) has been constructed by logging CAN traffic via the OBD-II port from a real vehicle while message injection attacks were performing. The classification goal is to distinguish cyber-attacks and normal samples by classifying the data samples. The dataset includes over 10,000 records and 10 attributes (including the target variable "Label").

### Variables
The definitions of the attributes are as followss.

* CAN ID : identifier of CAN message.
* DATA[0~7] : data value (byte), ranging from 0 to 255. They have been converted from hexadecimal numbers to decimal numbers.  
* Label : 0 indicates 'Normal', and 1 indicates an attack (including DoS, Fuzzy, Gear, or RPM).

## Question 1: Load Datasets (15pts)
A) Load the Dataset CAN-intrusion-dataset-10000.csv 

B) Split the data into equals-sized training and test sets (use a random_state = 1, and do not shuffle the data).  

C) How many observations do you have in your training set?  

D) How many observations for each class in your training set?

E) Z-standarize the input features of the training and test sets.

In [27]:
# Q1A) 
df = pd.read_csv('CAN-intrusion-dataset-10000.csv') # Reading the dataset using pandas

In [28]:
# Q1B) 
X = df.drop(columns=['Label']) # Using all attributes except 'Label' as predictors
y = df['Label'] # Predicting Label
# Splitting the data in equal-sized training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=1, shuffle=False)

In [29]:
# Q1C) 
# Using .shape to display the number of rows / observations in the dataset
train_observations_count = X_train.shape[0]
print("Number of observations in the training set:", train_observations_count)

Number of observations in the training set: 5000


In [30]:
### Q1D) 
# Using the .value_counts() function to find the number of normal samples and cyber-attack samples
train_class_counts = y_train.value_counts()
print("Number of observations for each class in the training set:\n", train_class_counts)

Number of observations for each class in the training set:
 Label
0    4284
1     716
Name: count, dtype: int64


In [31]:
### Q1E) 
# Using StandardScaler() to standardize features by removing the mean and scaling to unit variance
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Question 2: Logistic Regression (20pts)
A) Build a L1-regularized logistic regression model to all the training data, and then get the predicted labels for each item of the test set. 

B) Print out the precision, recall, and F1-score of the test set.

C) Print out the model execution time (including both training and testing time) in milliseconds. Please keep two decimal places.

In [ ]:
### Q2A)
# Start the timer
start_time = t.time()

# Initialize the Logistic Regression model with L1 penalty (as opposed to the default L2 penalty)
# 'liblinear' is a solver that is best suited for L1 regularization.
# A solver is an algorithm used to find the optimal parameters (weights) for the logistic regression model.
# Random state is clarified as 1
log_reg = LogisticRegression(penalty='l1', solver='liblinear', random_state=1)

# Training the model
log_reg.fit(X_train, y_train)

# Predicting the Label
y_pred = log_reg.predict(X_test)

# Stop the timer
end_time = t.time()

# Calculate execution time
execution_time = (end_time - start_time) * 1000

In [ ]:
### Q2B) 
# Find precision, recall, and F1 score
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Print precision, recall, and F1 score to 2 decimal places
print("Precision:", round(precision, 2))
print("Recall:", round(recall, 2))
print("F1 Score:", round(f1, 2))

Precision: 0.98
Recall: 0.56
F1 Score: 0.71


In [ ]:
### Q2C) 
# Print execution time to 2 decimal places
print("Execution Time (ms):", round(execution_time, 2))

Execution Time (ms): 91.33


## Question 3: Single Layer Neural Networks (30 pts)
In this task we aim to build models with better performance, using "deep" learning. __You may use PyTorch or Keras libraries for building deep learning models.__ 

A) Implement a single-layer neural network model that is used to classify the CAN intrusion data samples into normal and anomalous classes (0: normal, 1: attack). Use the standarized training set from Q1E) to train the network.

The details of the model are as follows:
* Use a Sigmoid as the output layer acitivation function to enable non-linearity. 
* Use the binary cross-entropy loss as a training criterion.
* Use Stochastic gradient descent optimizer with a learning rate of 0.1.
* Run the model for 10 iterations/epochs.

B) Record the loss for each iteration, and make a plot of iterations/epochs vs loss(Binary Cross Entropy).

C) Print out the precision, recall, and F1-score of the test set.

D) Print out the model execution time (including both training and testing time) in milliseconds. Please keep two decimal places.

In [35]:
### Q3A)Build and Train the Single-Layer Neural Network
start_time = time.time()  # Start timer for measuring execution time

# Define the model
model = Sequential()
model.add(Dense(1, activation='sigmoid', input_shape=(X_train_scaled.shape[1],)))

# Compile the model with specified settings
model.compile(optimizer=SGD(learning_rate=0.1), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model for 10 epochs
history = model.fit(X_train_scaled, y_train, epochs=10, batch_size=32, verbose=0)

end_training_time = time.time()  # Record training end time

# Evaluate the model on test data
loss, accuracy = model.evaluate(X_test_scaled, y_test, verbose=0)

end_testing_time = time.time()  # Record testing end time

In [36]:
### Q3B)Plot Loss for Each Epoch
loss_values = history.history['loss']
epochs = range(1, 11)
plt.plot(epochs, loss_values, marker='o', label='Binary Cross-Entropy Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Epochs vs Binary Cross-Entropy Loss')
plt.legend()
plt.show()


In [37]:
### Q3C) Calculate Precision, Recall, F1-Score
# Predict probabilities for the test set and convert to binary predictions
y_pred = (model.predict(X_test_scaled) > 0.5).astype(int)

# Calculate metrics
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

In [38]:
### Q3D)Calculate Execution Time
execution_time = (end_testing_time - start_time) * 1000  # Convert to milliseconds

# Print results
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1:.2f}")
print(f"Model Execution Time: {execution_time:.2f} ms")

## Question 4: Multi-Layer Perceptron (MLP) (35 pts)

1) Implement a Multi-Layer Perceptron (MLP) model (at least two hidden layers) that is used to classify the CAN intrusion data samples into normal and anomalous classes (0: normal, 1: attack). Use the standarized training set from Q1E) to train the network. 
The details of the model are as follows:
* Each hidden layer have 8 neurons/units. 
* Use tanh function as the activation function for hidden layers.
* Use a Sigmoid as the output layer acitivation function to enable non-linearity.  
* Use Stochastic gradient descent optimizer with a learning rate of 0.1.
* Run the model for 10 iterations/epochs 

B) Record the loss for each iteration, and make a plot of iterations/epochs vs loss(Binary Cross Entropy).

C) Print out the precision, recall, and F1-score of the test set.

D) Print out the model execution time (including both training and testing time) in milliseconds. Please keep two decimal places.

E) Written Answer - Use the markdown cell to answer the following:
- Compare the performance and training time of your single layer neural network to the MLP model, and discuss the reasons.

In [39]:
### Q4A)
import numpy as np
import time
from sklearn.metrics import precision_score, recall_score, f1_score
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD
import matplotlib.pyplot as plt

# Assuming standardized training and test sets are provided as X_train_std, X_test_std, y_train, y_test

# Record execution start time
start_time = time.time()

# A) Build Single-Layer Neural Network
model = Sequential()
model.add(Dense(1, activation='sigmoid', input_shape=(X_train_std.shape[1],)))

# Compile the model
optimizer = SGD(learning_rate=0.1)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Train the model for 10 epochs
history = model.fit(X_train_std, y_train, epochs=10, batch_size=32, verbose=0)

# Record execution end time
end_training_time = time.time()

# Evaluate the model on test data
_, accuracy = model.evaluate(X_test_std, y_test, verbose=0)

# Record test evaluation end time
end_testing_time = time.time()

# Total execution time in milliseconds
execution_time = (end_testing_time - start_time) * 1000

In [40]:
### Q4B)
#Record Loss and Plot Epochs vs Loss
loss = history.history['loss']
epochs = range(1, 11)
plt.plot(epochs, loss, marker='o', label='Binary Cross-Entropy Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Epochs vs Binary Cross-Entropy Loss')
plt.legend()
plt.show()

In [41]:
### Q4C)
# C) Calculate Precision, Recall, F1-Score
y_pred = (model.predict(X_test_std) > 0.5).astype(int)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Print results
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1:.2f}")

In [42]:
### Q4D)
#Print execution time in milliseconds
print(f"Model Execution Time: {execution_time:.2f} ms")

#### Q4E)  
Written answer here


## Question 5: Hyperparameter Optimization (10 Bonus pts)
A) Buil a Grid_Search_NN_model that has the same architecture as the MLP model from Question 4. Use grid search to tune two hyperparameters:
* The number of neurons on the hidden layers of your MLP model (find the best number among 8, 16, 32). Each hidden layer should have the same number of neurons/nodes, so only one hyperparameter is needed to tune the number of neurons.
* Learning rate of the SGD optimizer (find the best value among the two numbers 0.01 and 0.1). 

B) Implement grid search to identify optimal hyperparameter values, and print out the best hyperparameter values and the best cross-validation accuracy.

You can use 3-fold GridSearchCV and KerasClassifier functions on the standarized training set to do this. 

C) Build the optimized MLP model on the training set by passing the detected best hyperparameter values to the Grid_Search_NN_model. Print out the precision, recall, and F1-score of the optimized MLP model on the test set.

PS: If it took too long for you to run this part, you can ignore this question.

In [43]:
### Q5A)
def Grid_Search_NN_model(hidden_neurons = 8, learning_rate = 0.1):
    #write function here
    
    return myGSModel

In [44]:
### Q5B)
# Run gridsearch here

In [45]:
### Q5C)

## Make sure to add sufficient comments to your code, and run the entire code before submitting.